In [11]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing


In [12]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
endpoint = "https://docuement-intelligence-dev-eastus.cognitiveservices.azure.com/"
key = "c3f6b0bcf3be4852a268571d61229ffb"

# sample document
#formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"
formUrl = "https://www.emerson.com/documents/automation/instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788.pdf"

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

In [16]:
# models: prebuilt-document prebuilt-layout prebuilt-read

In [13]:
poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", formUrl)
result = poller.result()

In [14]:
result.content

'Instruction Manual D100399X012\nEW Valve February 2020\nFisher™ EWD, EWS, and EWT Valves through NPS 12x8\nContents\nIntroduction\n1\nScope of Manual\n1\nDescription\n3\nSpecifications\n3\nEducational Services\n4\nInstallation\n4\nInverted Globe Valve Applications\n(Actuator below valve)\n6\nMaintenance\n7\nPacking Lubrication\n8\nPacking Maintenance\n10\nReplacing Packing\n11\nTrim Maintenance\n14\nTrim Removal\n15\nLapping Metal Seats\n16\nValve Plug Maintenance\n17\nTrim Replacement\n19\nRetrofit: Installing C-seal Trim\n20\nReplacement of Installed C-seal Trim\n24\nTrim Removal (C-seal Constructions)\n24\nLapping Metal Seats (C-seal Constructions) . . 25\nRemachining Metal Seats\n(C-seal Constructions)\n25\nTrim Replacement (C-seal Constructions)\n26\nENVIRO-SEAL™ Bellows Seal Bonnet\n27\nParts Ordering\n32\nParts Kits\n33\nParts List\n36\nFigure 1. Fisher NPS 12x6 EW Series Valve With 667 Actuator\nFISHER\nATENB\n2\nW2777-2*\nIntroduction\nScope of Manual\nThis instruction manual

In [15]:
for table in result.tables:
    print(table.to_dict())

{'row_count': 29, 'column_count': 2, 'cells': [{'kind': 'content', 'row_index': 0, 'column_index': 0, 'row_span': 1, 'column_span': 2, 'content': 'Contents', 'bounding_regions': [{'page_number': 1, 'polygon': [{'x': 0.6745, 'y': 2.2641}, {'x': 4.0673, 'y': 2.2737}, {'x': 4.0673, 'y': 2.5814}, {'x': 0.6745, 'y': 2.5814}]}], 'spans': [{'offset': 104, 'length': 8}]}, {'kind': 'content', 'row_index': 1, 'column_index': 0, 'row_span': 1, 'column_span': 1, 'content': 'Introduction', 'bounding_regions': [{'page_number': 1, 'polygon': [{'x': 0.6745, 'y': 2.5814}, {'x': 3.7117, 'y': 2.5814}, {'x': 3.7117, 'y': 2.7496}, {'x': 0.6745, 'y': 2.7544}]}], 'spans': [{'offset': 113, 'length': 12}]}, {'kind': 'content', 'row_index': 1, 'column_index': 1, 'row_span': 1, 'column_span': 1, 'content': '1', 'bounding_regions': [{'page_number': 1, 'polygon': [{'x': 3.7117, 'y': 2.5814}, {'x': 4.0673, 'y': 2.5814}, {'x': 4.0673, 'y': 2.7496}, {'x': 3.7117, 'y': 2.7496}]}], 'spans': [{'offset': 126, 'length': 1

In [9]:
document_table = result.tables[1]

In [10]:
document_table

DocumentTable(row_count=15, column_count=3, cells=[DocumentTableCell(kind=columnHeader, row_index=0, column_index=0, row_span=1, column_span=1, content=Valve, bounding_regions=[BoundingRegion(page_number=4, polygon=[Point(x=0.7455, y=1.562), Point(x=2.4835, y=1.562), Point(x=2.4835, y=1.7273), Point(x=0.7455, y=1.7201)])], spans=[DocumentSpan(offset=6875, length=5)]), DocumentTableCell(kind=columnHeader, row_index=0, column_index=1, row_span=1, column_span=1, content=Seating, bounding_regions=[BoundingRegion(page_number=4, polygon=[Point(x=2.4835, y=1.562), Point(x=3.3309, y=1.562), Point(x=3.3309, y=1.7273), Point(x=2.4835, y=1.7273)])], spans=[DocumentSpan(offset=6881, length=7)]), DocumentTableCell(kind=columnHeader, row_index=0, column_index=2, row_span=1, column_span=1, content=Shutoff Class, bounding_regions=[BoundingRegion(page_number=4, polygon=[Point(x=3.3309, y=1.562), Point(x=7.726, y=1.5692), Point(x=7.726, y=1.7273), Point(x=3.3309, y=1.7273)])], spans=[DocumentSpan(offset

In [11]:
# Assuming 'document_table' is your input DocumentTable object
table_data = []

for row in range(document_table.row_count):
    row_data = []
    for column in range(document_table.column_count):
        cell = next((c for c in document_table.cells if c.row_index == row and c.column_index == column), None)
        cell_content = cell.content if cell else ""
        row_data.append(cell_content)
    table_data.append(row_data)

In [12]:
table_data

[['Valve', 'Seating', 'Shutoff Class'],
 ['EWD', 'Metal', 'II (standard)'],
 ['',
  '',
  'III (optional for NPS 6x4 through 12x6 valves with optional single graphite piston ring or for NPS 10x8 and 12×8 valves with optional double piston rings)'],
 ['',
  '',
  'IV (optional for NPS 6x4 through 12x8 valves with optional multiple graphite piston rings)'],
 ['EWS', 'Metal', 'IV (standard)'],
 ['', '', 'V (optional, consult your Emerson sales office)'],
 ['EWS', 'PTFE', 'VI'],
 ['EWT with all except Cavitrol III cages',
  'PTFE',
  'Standard Air Test (maximum leakage is 0.05 mL/min/psid/inch port diameter)'],
 ['', '', 'V (optional)'],
 ['', 'Metal', 'IV (standard)'],
 ['', 'Metal', 'V (optional)(1)'],
 ['EWT with 1-stage Cavitrol III cage', 'Metal', 'IV (standard)'],
 ['', '', 'V (optional)'],
 ['EWT with 2-stage Cavitrol III cage', 'Metal', 'V'],
 ['1. Class V shutoff for EWT requires spring loaded seal ring, radius-seat plug, wide-bevel seat ring, and seat lapping. Not available with 

In [26]:
for resulta in result.paragraphs: 
    print(resulta.content)

Instruction Manual D100399X012
EW Valve
February 2020
Fisher™ EWD, EWS, and EWT Valves through NPS 12x8
Contents
Introduction
1
Scope of Manual
1
Description
3
Specifications
3
Educational Services
4
Installation
4
Inverted Globe Valve Applications
(Actuator below valve)
6
Maintenance
7
Packing Lubrication
8
Packing Maintenance
10
Replacing Packing
11
Trim Maintenance
14
Trim Removal
15
Lapping Metal Seats
16
Valve Plug Maintenance
17
Trim Replacement
19
Retrofit: Installing C-seal Trim
20
Replacement of Installed C-seal Trim
24
Trim Removal (C-seal Constructions)
24
Lapping Metal Seats (C-seal Constructions) . .
Remachining Metal Seats
25
(C-seal Constructions)
25
Trim Replacement (C-seal Constructions)
26
ENVIRO-SEAL™ Bellows Seal Bonnet
27
Parts Ordering
32
Parts Kits
33
Parts List
36
Figure 1. Fisher NPS 12x6 EW Series Valve With 667 Actuator
FISHER 2
ATENB
W2777-2*
Introduction
Scope of Manual
This instruction manual includes installation, maintenance, and parts information for Fi

In [40]:
for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
         "handwritten" if style.is_handwritten else "no handwritten"
        )
    )

Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content


In [41]:
result.paragraphs

[DocumentParagraph(role=None, content=Instruction Manual D100399X012, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.7774, y=0.7161), Point(x=2.2348, y=0.7114), Point(x=2.2359, y=1.024), Point(x=0.7784, y=1.0288)])], spans=[DocumentSpan(offset=0, length=30)]),
 DocumentParagraph(role=None, content=EW Valve, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=7.1008, y=0.7257), Point(x=7.7025, y=0.7257), Point(x=7.6978, y=0.8641), Point(x=7.1008, y=0.8594)])], spans=[DocumentSpan(offset=31, length=8)]),
 DocumentParagraph(role=None, content=February 2020, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=7.0006, y=0.9119), Point(x=7.6978, y=0.9071), Point(x=7.6978, y=1.036), Point(x=7.0006, y=1.0408)])], spans=[DocumentSpan(offset=40, length=13)]),
 DocumentParagraph(role=None, content=Fisher™ EWD, EWS, and EWT Valves through NPS 12x8, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.7449, y=1.4036), Point(x=5.7256, y=1.4

In [37]:
for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
        table_idx, table.row_count, table.column_count
        )
    )

    # Assuming 'document_table' is your input DocumentTable object
    table_data = []

    for row in range(table.row_count):
        row_data = []
        for column in range(table.column_count):
            cell = next((c for c in table.cells if c.row_index == row and c.column_index == column), None)
            cell_content = cell.content if cell else ""
            row_data.append(cell_content)
        table_data.append(row_data)

    break
        

Table # 0 has 29 rows and 2 columns


In [38]:
table_data

[['Contents', ''],
 ['Introduction', '1'],
 ['Scope of Manual', '1'],
 ['Description', '3'],
 ['Specifications', '3'],
 ['Educational Services', '4'],
 ['Installation', '4'],
 ['Inverted Globe Valve Applications', ''],
 ['(Actuator below valve)', '6'],
 ['Maintenance', '7'],
 ['Packing Lubrication', '8'],
 ['Packing Maintenance', '10'],
 ['Replacing Packing', '11'],
 ['Trim Maintenance', '14'],
 ['Trim Removal', '15'],
 ['Lapping Metal Seats', '16'],
 ['Valve Plug Maintenance', '17'],
 ['Trim Replacement', '19'],
 ['Retrofit: Installing C-seal Trim', '20'],
 ['Replacement of Installed C-seal Trim', '24'],
 ['Trim Removal (C-seal Constructions)', '24'],
 ['Lapping Metal Seats (C-seal Constructions) .', '. 25'],
 ['Remachining Metal Seats', ''],
 ['(C-seal Constructions)', '25'],
 ['Trim Replacement (C-seal Constructions)', '26'],
 ['ENVIRO-SEAL™ Bellows Seal Bonnet', '27'],
 ['Parts Ordering', '32'],
 ['Parts Kits', '33'],
 ['Parts List', '36']]

In [42]:
for page in result.pages:
    for line_idx, line in enumerate(page.lines):
        print(
         "...Line # {} has text content '{}'".format(
        line_idx,
        line.content.encode("utf-8")
        )
    )


...Line # 0 has text content 'b'Instruction Manual''
...Line # 1 has text content 'b'D100399X012''
...Line # 2 has text content 'b'EW Valve''
...Line # 3 has text content 'b'February 2020''
...Line # 4 has text content 'b'Fisher\xe2\x84\xa2 EWD, EWS, and EWT Valves''
...Line # 5 has text content 'b'through NPS 12x8''
...Line # 6 has text content 'b'Contents''
...Line # 7 has text content 'b'Introduction''
...Line # 8 has text content 'b'1''
...Line # 9 has text content 'b'Scope of Manual''
...Line # 10 has text content 'b'1''
...Line # 11 has text content 'b'Description''
...Line # 12 has text content 'b'3''
...Line # 13 has text content 'b'Specifications''
...Line # 14 has text content 'b'3''
...Line # 15 has text content 'b'Educational Services''
...Line # 16 has text content 'b'4''
...Line # 17 has text content 'b'Installation''
...Line # 18 has text content 'b'4''
...Line # 19 has text content 'b'Inverted Globe Valve Applications''
...Line # 20 has text content 'b'(Actuator below v

In [43]:
for selection_mark in page.selection_marks:
    print(
        "...Selection mark is '{}' and has a confidence of {}".format(
        selection_mark.state,
        selection_mark.confidence
        )
    )

In [35]:
for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
        table_idx, table.row_count, table.column_count
        )
    )
        
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
            cell.row_index,
            cell.column_index,
            cell.content.encode("utf-8"),
            )
        )

Table # 0 has 29 rows and 2 columns
...Cell[0][0] has content 'b'Contents''
...Cell[1][0] has content 'b'Introduction''
...Cell[1][1] has content 'b'1''
...Cell[2][0] has content 'b'Scope of Manual''
...Cell[2][1] has content 'b'1''
...Cell[3][0] has content 'b'Description''
...Cell[3][1] has content 'b'3''
...Cell[4][0] has content 'b'Specifications''
...Cell[4][1] has content 'b'3''
...Cell[5][0] has content 'b'Educational Services''
...Cell[5][1] has content 'b'4''
...Cell[6][0] has content 'b'Installation''
...Cell[6][1] has content 'b'4''
...Cell[7][0] has content 'b'Inverted Globe Valve Applications''
...Cell[7][1] has content 'b'''
...Cell[8][0] has content 'b'(Actuator below valve)''
...Cell[8][1] has content 'b'6''
...Cell[9][0] has content 'b'Maintenance''
...Cell[9][1] has content 'b'7''
...Cell[10][0] has content 'b'Packing Lubrication''
...Cell[10][1] has content 'b'8''
...Cell[11][0] has content 'b'Packing Maintenance''
...Cell[11][1] has content 'b'10''
...Cell[12][0] ha

In [44]:
for kv_pair in result.key_value_pairs:
    if kv_pair.key and kv_pair.value:
        print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
    else:
        print("Key '{}': Value:".format(kv_pair.key.content))

In [45]:
def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])

In [46]:
print ("Document contains content: ", result.content)

for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
            "handwritten" if style.is_handwritten else "no handwritten"
        )
    )

for page in result.pages:
    print("----Analyzing Read from page #{}----".format(page.page_number))
    print(
        "Page has width: {} and height: {}, measured with unit: {}".format(
            page.width, page.height, page.unit
        )
    )

    for line_idx, line in enumerate(page.lines):
        print(
            "...Line # {} has text content '{}' within bounding box '{}'".format(
                line_idx,
                line.content,
                format_bounding_box(line.polygon),
            )
        )

    for word in page.words:
        print(
            "...Word '{}' has a confidence of {}".format(
                word.content, word.confidence
            )
        )

print("----------------------------------------")

Document contains content:  Instruction Manual D100399X012
EW Valve
February 2020
Fisher™ EWD, EWS, and EWT Valves through NPS 12x8
Contents
Introduction
1
Scope of Manual
1
Description
3
Specifications
3
Educational Services
4
Installation
4
Inverted Globe Valve Applications
(Actuator below valve)
6
Maintenance
7
Packing Lubrication
8
Packing Maintenance
10
Replacing Packing
11
Trim Maintenance
14
Trim Removal
15
Lapping Metal Seats
16
Valve Plug Maintenance
17
Trim Replacement
19
Retrofit: Installing C-seal Trim
20
Replacement of Installed C-seal Trim
24
Trim Removal (C-seal Constructions)
24
Lapping Metal Seats (C-seal Constructions) . .
Remachining Metal Seats
25
(C-seal Constructions)
25
Trim Replacement (C-seal Constructions)
26
ENVIRO-SEAL™ Bellows Seal Bonnet
27
Parts Ordering
32
Parts Kits
33
Parts List
36
Figure 1. Fisher NPS 12x6 EW Series Valve With 667 Actuator
FISHER 2
ATENB
W2777-2*
Introduction
Scope of Manual
This instruction manual includes installation, maintenance, 

In [1]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader


In [7]:
AzureAIDocumentIntelligenceLoader?

Init signature:
AzureAIDocumentIntelligenceLoader(
    api_endpoint: str,
    api_key: str,
    file_path: Optional[str] = None,
    url_path: Optional[str] = None,
    api_version: Optional[str] = None,
    api_model: str = 'prebuilt-layout',
    mode: str = 'markdown',
) -> None
Docstring:      Loads a PDF with Azure Document Intelligence
Init docstring:
Initialize the object for file processing with Azure Document Intelligence
(formerly Form Recognizer).

This constructor initializes a AzureAIDocumentIntelligenceParser object to be
used for parsing files using the Azure Document Intelligence API. The load
method generates Documents whose content representations are determined by the
mode parameter.

Parameters:
-----------
api_endpoint: str
    The API endpoint to use for DocumentIntelligenceClient construction.
api_key: str
    The API key to use for DocumentIntelligenceClient construction.
file_path : Optional[str]
    The path to the file that needs to be loaded.
    Either file_

In [16]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

file_path = formUrl
endpoint = "https://docuement-intelligence-dev-eastus.cognitiveservices.azure.com/"
key = "c3f6b0bcf3be4852a268571d61229ffb"
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, url_path=file_path, api_model="prebuilt-layout", mode="markdown"
)

documents = loader.load()

In [18]:
documents

[Document(page_content='<!-- PageHeader="Instruction Manual D100399X012" -->\n\n<!-- PageHeader="EW Valve February 2020" -->\n\nFisher™ EWD, EWS, and EWT Valves through NPS 12x8\n===\n\n|||\n| - | - |\n| Contents ||\n| Introduction | 1 |\n| Scope of Manual | 1 |\n| Description | 3 |\n| Specifications | 3 |\n| Educational Services | 4 |\n| Installation | 4 |\n| Inverted Globe Valve Applications | |\n| \\(Actuator below valve\\) | 6 |\n| Maintenance | 7 |\n| Packing Lubrication | 8 |\n| Packing Maintenance | 10 |\n| Replacing Packing | 11 |\n| Trim Maintenance | 14 |\n| Trim Removal | 15 |\n| Lapping Metal Seats | 16 |\n| Valve Plug Maintenance | 17 |\n| Trim Replacement | 19 |\n| Retrofit: Installing C\\-seal Trim | 20 |\n| Replacement of Installed C\\-seal Trim | 24 |\n| Trim Removal \\(C\\-seal Constructions\\) | 24 |\n| Lapping Metal Seats \\(C\\-seal Constructions\\) \\. | 25 \\. |\n| Remachining Metal Seats | |\n| \\(C\\-seal Constructions\\) | 25 |\n| Trim Replacement \\(C\\-seal 